In [ ]:
import torch
import torchvision
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from collections import Counter
from tqdm import tqdm
import PIL 
from PIL import Image
import pandas as pd
import datetime

In [ ]:
import pandas as pd

Data = pd.read_csv('/content/cartoon_set/labels.csv', delim_whitespace= True)
Data.set_index('file_name', inplace=True)

In [ ]:
import glob
import shutil
PATH = '/content/cartoon_set/img/'
filenames = glob.glob(PATH + '*png')
print(len(filenames))

In [ ]:
for i in ['train', 'valid']:
  for j in ['0','1','2','3','4']:
    if not os.path.exists(os.path.join(PATH, i, j)):
      os.makedirs(os.path.join(PATH, i, j))

In [ ]:
i = 0
j = 0
shuffle = np.random.permutation(len(filenames))
for j in tqdm(shuffle[:9000]):
  file = filenames[j].split('/')[-1]
  #print(file)
  if Data[Data.index == file]['face_shape'].item() == 0:
    shutil.copy(PATH + file, PATH + 'train/' +'0/' + file)
  
  elif Data[Data.index == file]['face_shape'].item() == 1:
    shutil.copy(PATH + file, PATH + 'train/' +'1/' + file)

  elif Data[Data.index == file]['face_shape'].item() == 2:
    shutil.copy(PATH + file, PATH + 'train/' +'2/' + file)

  elif Data[Data.index == file]['face_shape'].item() == 3:
    shutil.copy(PATH + file, PATH + 'train/' +'3/' + file)

  elif Data[Data.index == file]['face_shape'].item() == 4:
    shutil.copy(PATH + file, PATH + 'train/' +'4/' + file)

for i in tqdm(shuffle[9000:]):
  file = filenames[i].split('/')[-1]
  #print(file)
  if Data[Data.index == file]['face_shape'].item() == 0:
    shutil.copy(PATH + file, PATH + 'valid/' +'0/' + file)
  
  elif Data[Data.index == file]['face_shape'].item() == 1:
    shutil.copy(PATH + file, PATH + 'valid/' +'1/' + file)

  elif Data[Data.index == file]['face_shape'].item() == 2:
    shutil.copy(PATH + file, PATH + 'valid/' +'2/' + file)

  elif Data[Data.index == file]['face_shape'].item() == 3:
    shutil.copy(PATH + file, PATH + 'valid/' +'3/' + file)

  elif Data[Data.index == file]['face_shape'].item() == 4:
    shutil.copy(PATH + file, PATH + 'valid/' +'4/' + file)

In [ ]:
data_dir = '/content/cartoon_set/img/'
input_size = 400
batch_size = 10
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(224),
        transforms.ColorJitter(brightness=0, contrast=0.7, saturation=0.7, hue=0),
        transforms.Grayscale(num_output_channels=3),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomRotation(degrees=15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.Grayscale(num_output_channels=3),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=7) for x in ['train', 'valid']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
trainiter = iter(dataloaders_dict['train'])
features, labels = next(trainiter)
features.shape, labels.shape

In [ ]:
categories = []
for d in os.listdir('/content/cartoon_set/img/train/'):
    categories.append(d)
    
n_classes = len(categories)
print(f'There are {n_classes} different classes.')

In [ ]:
class_to_idx = image_datasets['train'].class_to_idx
idx_to_class = {
    idx: class_
    for class_, idx in image_datasets['train'].class_to_idx.items()
}


train_cnts = Counter([idx_to_class[x] for x in image_datasets['train'].targets])
val_cnts = Counter([idx_to_class[x] for x in  image_datasets['valid'].targets])

train_cnts = pd.DataFrame({'cat' :list(train_cnts.keys()), 'train_cnt': list(train_cnts.values())})
val_cnts = pd.DataFrame({'cat' :list(val_cnts.keys()), 'val_cnt': list(val_cnts.values())})


cnt_df = pd.merge(train_cnts,val_cnts,on='cat',how='left')

In [ ]:
from torchvision.utils import make_grid
plt.imshow(make_grid(features).permute(1, 2, 0))
plt.show()
print(labels)

In [ ]:
from torchvision import models
model = models.vgg16(pretrained=True)
model2 = models.resnet50(pretrained=True)

for param in model.parameters():

  param.requires_grad = False

# for param in model.classifier[6].parameters():
#   param.requires_grad = False


for param in model2.parameters():
  param.requires_grad = True

In [ ]:
n_inputs = model.classifier[6].in_features

In [ ]:
num_inputs = model2.fc.in_features


model2.fc = nn.Sequential(nn.Linear(num_inputs, n_classes))

model2 = model2.to(device)

In [ ]:
import torch.nn as nn




model.classifier[6] = nn.Sequential(
    nn.Linear(n_inputs, 2048), nn.ReLU(), nn.Dropout(0.65),
    nn.Linear(2048, 1024),nn.ReLU(),
    nn.Dropout(0.6),
    nn.Linear(1024, 512),nn.ReLU(),
    nn.Linear(512, n_classes))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model= model.to(device)

In [ ]:
from torch import optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.01,momentum = 0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer,3,verbose=True, gamma=0.1)

In [ ]:
from timeit import default_timer as timer
def train(model,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          save_file_name,
          scheduler,
          max_epochs_stop=3,
          n_epochs=30,
          print_every=1,
          ):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in tqdm(range(n_epochs)):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()

       

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            data, target = data.cuda(), target.cuda()

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(data)

            # Loss and backpropagation of gradients
            loss = criterion(output, target)
            loss.backward()

            # Update the parameters
            optimizer.step()
            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)



            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    data, target = data.cuda(), target.cuda()

                    # Forward pass
                    output = model(data)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)
                scheduler.step()

                

                  

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )


                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

In [ ]:
def crossvalid(model=None, criterion=None, optimizer=None, dataset=None, k_fold=5):
    train_score = pd.Series()
    val_score = pd.Series()

    total_size = len(dataset)
    fraction = 1 / k_fold
    seg = int(total_size * fraction)
    # tr:train,val:valid; r:right,l:left;  eg: trrr: right index of right side train subset
    # index: [trll,trlr],[vall,valr],[trrl,trrr]
    for i in range(k_fold):
        trll = 0
        trlr = i * seg
        vall = trlr
        valr = i * seg + seg
        trrl = valr
        trrr = total_size
        # msg
        #         print("train indices: [%d,%d),[%d,%d), test indices: [%d,%d)"
        #               % (trll,trlr,trrl,trrr,vall,valr))

        train_left_indices = list(range(trll, trlr))
        train_right_indices = list(range(trrl, trrr))

        train_indices = train_left_indices + train_right_indices
        val_indices = list(range(vall, valr))

        train_set = torch.utils.data.dataset.Subset(dataset, train_indices)
        val_set = torch.utils.data.dataset.Subset(dataset, val_indices)

        #         print(len(train_set),len(val_set))
        #         print()

        train_loader = torch.utils.data.DataLoader(train_set, batch_size=50,
                                                   shuffle=True, num_workers=4)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=50,
                                                 shuffle=True, num_workers=4)
        train_acc = train(res_model, criterion, optimizer, train_loader, epoch=1)
        train_score.at[i] = train_acc
        val_acc = valid(res_model, criterion, optimizer, val_loader)
        val_score.at[i] = val_acc

    return train_score, val_score


In [ ]:
batch_sizes = [8]
learning_rates = [0.001]
momentums = [0.9]
model_list =['resnet']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
output_dict={}
num_epochs = 10

for batch_size in batch_sizes:
  for learning_rate in learning_rates:
    for momentum in momentums:
      for model_choice in model_list:
      
        dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=5) for x in ['train', 'valid']}

        if model_choice == 'vgg':
          model = models.vgg16(pretrained=True)

          for param in model.parameters():

            param.requires_grad = False

          n_inputs = model.classifier[6].in_features
          model.classifier[6] = nn.Sequential(
          nn.Linear(n_inputs, 2048), nn.ReLU(), nn.Dropout(0.65),
          nn.Linear(2048, 1024),nn.ReLU(),
          nn.Dropout(0.6),
          nn.Linear(1024, 512),nn.ReLU(),
          nn.Linear(512, n_classes))

          model = model.to(device)


        if model_choice == 'resnet':

          model = models.resnet50(pretrained=True)

          for param in model.parameters():

            param.requires_grad = True

          num_inputs = model.fc.in_features
          model.fc = nn.Sequential(nn.Linear(num_inputs, n_classes))
          model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.01,momentum = momentum)
        exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer,3,gamma=1)


        NEWPATH = '/content/drive/My Drive/'
        save_path = os.path.join(NEWPATH + 'ResnetB1.pth')
        checkpoint_path = NEWPATH

        model,history = train(model,criterion,optimizer,dataloaders_dict['train'],
              dataloaders_dict['valid'],
              save_path,
              exp_lr_scheduler,
              max_epochs_stop = 10, n_epochs = num_epochs,
              )
        
        print('')
        print('Batch Size', batch_size, 'Learning_rate', learning_rate)
        print('')
        print('Momentum', momentum, 'model', model_choice)


        plt.figure(figsize=(8, 6))
        for c in ['train_loss', 'valid_loss']:
            plt.plot(
                history[c], label=c)
        plt.legend()
        plt.xlabel('Epoch')
        plt.ylabel('Average Loss')
        plt.title('Training and Validation Losses')
        plt.savefig('/content/drive/My Drive/B1_loss' + str(batch_size) + str(learning_rate)+ str(momentum)+ model_choice + '.png')


        plt.figure(figsize=(8, 6))
        for c in ['train_acc', 'valid_acc']:
            plt.plot(
                100 * history[c], label=c)
        plt.legend()
        plt.xlabel('Epoch')
        plt.ylabel('Average Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.savefig('/content/drive/My Drive/B1_acc' + str(batch_size) + str(learning_rate)+ str(momentum)+ model_choice + '.png')


        output_dict[str(batch_size) + str(learning_rate) + str(momentum) + model_choice] = history

        torch.cuda.empty_cache()
